In [ ]:
from collections import OrderedDict
from best.hVI_fsrm import SPaths
from best.hVI_fsrm import spec_Spaths
from best.hVI_models import State_Space, Det_SI_Model
from best.hVI_types import Env, Gamma
import best.rss18_functions as rf
from best.hVI_config import sc, load, parr, obs_action, epsilon, rand_seed
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import copy
import cPickle as pkl
import random
from os import fork
from joblib import Parallel, delayed
import multiprocessing
from itertools import product
import time

from polytope import *


In [2]:
import sys

In [3]:
print "Setting Up Scenario"

# Define Regions
# Regs have the same format as RSS code. Regs that are added first have a higher priority
#### Wall region
print("Started wall case")
regs = OrderedDict()
p1 = rf.vertex_to_poly(np.array([[2, 0], [3, 0], [3, 1], [2, 1]]))
regs['r1'] = (p1, 1, 'obs')
p2 = rf.vertex_to_poly(np.array([[2, 1], [3, 1], [3, 2], [2, 2]]))
regs['r2'] = (p2, 1, 'obs')
p3 = rf.vertex_to_poly(np.array([[2, 2], [3, 2], [3, 3], [2, 3]]))
regs['r3'] = (p3, 1, 'obs')
p4 = rf.vertex_to_poly(np.array([[2, 3], [3, 3], [3, 4], [2, 4]]))
regs['r4'] = (p4, .6, 'obs', 1)
p5 = rf.vertex_to_poly(np.array([[2, 4], [3, 4], [3, 5], [2, 5]]))
regs['r5'] = (p5, .6, 'obs', 0)

p1 = rf.vertex_to_poly(np.array([[2, 0], [3, 0], [3, -1], [2, -1]]))
regs['r6'] = (p1, 1, 'obs')
p2 = rf.vertex_to_poly(np.array([[2, -1], [3, -1], [3, -2], [2, -2]]))
regs['r7'] = (p2, 1, 'obs')
p3 = rf.vertex_to_poly(np.array([[2, -2], [3, -2], [3, -3], [2, -3]]))
regs['r8'] = (p3, 1, 'obs')
p4 = rf.vertex_to_poly(np.array([[2, -3], [3, -3], [3, -4], [2, -4]]))
regs['r9'] = (p4,1, 'obs', 0)
p5 = rf.vertex_to_poly(np.array([[2, -4], [3, -4], [3, -5], [2, -5]]))
regs['r10'] = (p5, .9, 'obs', 0)

a1 = rf.vertex_to_poly(np.array([[4, 0], [5, 0], [5, 1], [4, 1]]))
regs['a1'] = (a1, 0.9, 'sample1', 1)

a2 = rf.vertex_to_poly(np.array([[4, -3], [5, -3], [5, -2], [4, -2]]))
regs['a2'] = (a2, 0.1, 'sample2', 1)

output_color = {'r1': 'red', 'r2': 'red', 'r3': 'red', 'r4': 'red', 'r5': 'red', 'r6': 'red', 'r7': 'red',
                'r8': 'red', 'r9': 'red', 'r10': 'red',
                'a1': 'green','a2': 'blue', 'null': 'white'}
# Define Null regions with bounds of the space for null output with lowest priority
p = rf.vertex_to_poly(np.array([[-3, -5], [-3, 5], [5, -5], [5, 5]]))
regs['null'] = (p, 1.0, 'null')

# Construct belief-MDP for env
env = Env(regs)
print(env)

Setting Up Scenario
Started wall case
Regions: 
OrderedDict([('r4', 0), ('r5', 1), ('r10', 2), ('a1', 3), ('a2', 4)])

Total number of unknown regions: 5
Total number of  regions: 13


In [5]:
''' Configuration Parameters '''
random.seed(rand_seed)
np.random.seed(rand_seed)

print ''' Setup Motion and Observation Models '''
# Define Motion and Observation Model
Wx = np.eye(2)
Wu = np.eye(2)
r2_bs = State_Space([-5, -5], [5, 5])
motion_model = Det_SI_Model(0.1)

print " Constructing FIRM"
fig = plt.figure(figsize=(14, 14), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, aspect='equal')
prm = SPaths(r2_bs, motion_model, Wx, Wu, regs, output_color, ax)
prm.make_nodes_edges(40, 3, init=np.array([[-4.5],[0]]))
t1 = time.clock()
# prm.compute_output_prob()
t2 = time.clock()
prm.plot(ax)
plt.show()

 Setup Motion and Observation Models 
 The used model contains an integrator for each dimension:

[[1. 0.]
 [0. 1.]]
 Constructing FIRM
[[1. 0.]
 [0. 1.]]
[[0.1 0. ]
 [0.  0.1]]
[[10.51249  0.     ]
 [ 0.      10.51249]]


In [9]:
print('-- Generate the DFA and the Product model c')
from best.fsa import Fsa
props = ['obs', 'sample1', 'sample2']
props = dict(zip(props, map(lambda x: 2 ** x, range(0, len(props)))))
print(props)
fsa = Fsa()
vars(Fsa)
fsa.g.add_node(0)
fsa.g.add_node('trap')
fsa.g.add_node(1)

fsa.g.add_edge(0,1, weight=0, input={props['sample1'],props['sample2'],props['sample2']+props['sample1']})
fsa.g.add_edge(0,0, weight=0, input={0})
fsa.g.add_edge(0,'trap', weight=0, input={props['obs']})

fsa.props=props
fsa.final = {1}
fsa.init = dict({0:1})

#fsa.g.add_edge(src, dest, weight=0, input=bitmaps, guard=guard, label=guard)
from best.ltl import formula_to_mdp

fsaform = Fsa()
form = '! obs U sample'
fsaform.from_formula(form)
vars(fsa.g)
print(fsaform.g.edges(data=True))
formula_fsa = dict()
formula_fsa['fsa'] = fsa
formula_fsa['init'] = dict({0:1})
formula_fsa['final'] = {1}
formula_fsa['prop'] = props

env.get_prop('r3')
prod_ = spec_Spaths(prm, formula_fsa,env,n=125)


-- Generate the DFA and the Product model c
{'sample1': 2, 'obs': 1, 'sample2': 4}
[(u'1', u'1', {'input': set([0, 1, 2, 3]), 'guard': '(1)', 'weight': 0, 'label': '(1)'}), (u'0', u'1', {'input': set([1, 3]), 'guard': u'sample', 'weight': 0, 'label': u'sample'}), (u'0', u'0', {'input': set([0]), 'guard': u'(!sample & !obs)', 'weight': 0, 'label': u'(!sample & !obs)'})]


In [11]:
import matplotlib.pyplot as plt


def plot_bel(n,i):
    print(prod_.env.b_reg_init.tolist())
    
    (q,v) =  n
    vals = []
    for b_i in np.linspace(0,1,20):
        b_ind = [ prob if not el == i else [b_i]  for el,prob in enumerate(prod_.env.b_reg_init.tolist()) ]
        b = prod_.env.get_product_belief(b_ind)
        vals += [max(prod_.val[(q,v)].alpha_mat.T * b).tolist()]
    plt.figure(0)

    plt.plot(np.linspace(0,1,20),sum(vals,[]))
    plt.ylabel('probability')
    ax = plt.gca()
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    plt.show()

    



In [15]:
print('Start Back-ups')

not_converged = True
i = 0
(q,key) = prod_.init[0]
n = (q,20) 
opts =dict()
while not_converged:
    print('iteration', i)
    not_converged = prod_.full_back_up(opts)
    opt = np.unique(prod_.val[n].best_edge) 
    if i>20:
        not_converged = False
    i += 1

Start Back-ups
('iteration', 0)


KeyError: (0, 20)

In [9]:

from best.hVI_fsrm import plot_optimizers, simulate
import imp
imp.reload(F)
plot_optimizers(prod_,ax)

best.hVI_fsrm.simulate.simulate(prod_, regs)


NameError: name 'F' is not defined

In [10]:
''' Execute the policy '''
b = prod_.env.b_prod_init
plt.figure()
print b
print(prod_.init)
(q,v) = prod_.init[0]
# v = 0
traj = []
v_list =[v]
act_list =[]
vals = []
for t in range(150):
    # import pdb; pdb.set_trace()
    #print((q,v))
    #print(prod_.val[(q,v)].alpha_mat.T * np.array(b))
    #print "val = " + str(max(prod_.val[(q,v)].alpha_mat.T * b))
    #vals += [str(max(prod_.val[(q,v)].alpha_mat.T * b))]
    # Get best edge
    alpha_new, best_e, opt = prod_._back_up(q, v, b)
    # print("best action = ",best_e)
    vals += [opt]
#     i_best_alpha = np.argmax(prod_.val[(q,v)].alpha_mat.T * b)
#     best_e = prod_.val[(q,v)].best_edge[i_best_alpha]

    if obs_action is True and best_e < 0:
        reg_key = prod_.env.regs.keys()[-1 * (best_e + 1)]
        (b_, o, i_o) = env.get_b_o_reg(b, env.regs[reg_key][3], reg_key, v.mean)
        b = b_
        act_list += ["Observing = " +reg_key ]
        print "Observing " +reg_key + " at vertex" + str(v) + " q = " + str(q) 
        continue
    # Simulate trajectory under edges
    edge_controller = prm.edge_controllers[v][prm.edges[v].index(best_e)]
    traj_e = edge_controller.simulate_trajectory(edge_controller.node_i)
    traj_n = prm.node_controllers[prm.nodes.index(edge_controller.node_j)].simulate_trajectory(traj_e[-1])
    # traj_i = [(b, i, q) for i in traj_e + traj_n]
    traj_i = traj_e + traj_n
    traj = traj + traj_i
    # Get q', v', q' and loop
    z = prm.get_output_traj(traj_e + traj_n)
    v_ = best_e
    act_list += [best_e]

    if obs_action is False:
        print "TODO: Implement backup without obs_action"
    else:
        if regs[z][2] is 'null':
            b_ = b
            q_ = q
        elif regs[z][2] is 'obs' or regs[z][2] is 'sample1':
            q_ = None
            b_ = None
            # if region is known
            if regs[z][1] == 1 or regs[z][1] == 0:
                # if label is true
                if regs[z][1] == 1:
                    q_ = prod_.fsa.next_states_of_fsa(q,regs[z][2]) 
                    b_ = b
            # else update belief by simulating an observation
            else:
                (b_, o, i_o) = env.get_b_o_reg(b, env.regs[z][3], z)
                # if true label is true then do transition in DFA
                # We are assuming that we get zero false rate when we pass through the region
                #print(regs[z][2], i_o)
                if regs[z][3] is 1:
                    #print(regs[z][2], regs[z][3])
                    q_ = prod_.fsa.next_states_of_fsa(q,(env.regs[z][2],)) 
            if q_ is None:
                q_ = q
            if b_ is None:
                b_ = b
    print "going from vertex " + str(v) + " to vertex " + str(v_) + " q = " + str(q)

    b = b_
    q = q_
    v = v_
    v_list += [v]
    if q in list(prod_.dfsa_final) or q == list(prod_.dfsa_final):
        print('break')
        break
print(v_list)
print(vals)
print(act_list)
%matplotlib inline
f =prod_.firm.plot_traj(traj, 'green')
plt.show()

[[0.00144]
 [0.00216]
 [0.00216]
 [0.00324]
 [0.01296]
 [0.01944]
 [0.01944]
 [0.02916]
 [0.01296]
 [0.01944]
 [0.01944]
 [0.02916]
 [0.11664]
 [0.17496]
 [0.17496]
 [0.26244]
 [0.00016]
 [0.00024]
 [0.00024]
 [0.00036]
 [0.00144]
 [0.00216]
 [0.00216]
 [0.00324]
 [0.00144]
 [0.00216]
 [0.00216]
 [0.00324]
 [0.01296]
 [0.01944]
 [0.01944]
 [0.02916]]
[(0, <node at x:-4.5 y:0.0>)]
going from vertex [[-4.5]
 [ 0. ]] to vertex [[-2.16172]
 [ 1.06083]] q = 0
going from vertex [[-2.16172]
 [ 1.06083]] to vertex [[-0.2877 ]
 [ 3.16168]] q = 0
going from vertex [[-0.2877 ]
 [ 3.16168]] to vertex [[2.5]
 [3.5]] q = 0


TypeError: unhashable type: 'list'

In [11]:
plt.show()

ValueError: max() arg is an empty sequence